In [1]:
import numpy as np
import pandas as pd
import os, pickle

from sklearn.manifold import TSNE
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
mapping = pd.read_pickle('/home/jovyan/shared-scratch/jhoon/ocp2023/oc20dense_metadata/oc20dense_mapping.pkl')

In [2]:
model = 'gnoc'
data_path = f'/home/jovyan/shared-scratch/jhoon/ocp2023/clip_data/oc20dense_eval_{model}_relaxed.pkl'
df_data = pd.read_pickle(data_path)

# Adsorbate

In [8]:
sid_list, ads_list = [], []
for i in range(len(df_data)):
    sid = df_data['id'].iloc[i].split('_')[0]
    ads = mapping[int(sid)]['adsorbate']
    sid_list.append(sid)
    ads_list.append(ads)
df_data['sid'] = sid_list
df_data['ads'] = ads_list
df_data = df_data.drop(columns=['text', 'chg_emb'])

In [21]:
def classify_adsorbate(adsorbate):
    """
    Classify adsorbates based on the number of C and N atoms.
    """
    c_count = adsorbate.count('C')
    n_count = adsorbate.count('N')

    if n_count == 2:
        return 'N2'   # With 2 N
    elif n_count == 1:
        return 'N1'   # With 1 N
    elif c_count == 2:
        return 'C2'   # With 2 C
    elif c_count == 1:
        return 'C1'   # With 1 C
    elif c_count == 0 and n_count == 0:
        return 'O&H'  # Without C & N
    else:
        return 'Other' # Other compositions

In [22]:
df_data['category'] = df_data['ads'].apply(classify_adsorbate)

In [23]:
category_counts = df_data['category'].value_counts()

category_counts

C2     4892
C1     4507
O&H    1483
N2      626
Name: category, dtype: int64

In [19]:
df_data

,id,target,sid,ads,category
0,85487_-1,-0.559899,85487,*CH2*O,C1
1,39118_110,-1.130786,39118,*O,O&H
2,2797_-1,-1.337591,2797,*OCH3,C1
3,64221_170,0.885938,64221,*CH,C1
4,44017_160,-2.090978,44017,*CH,C1
...,...,...,...,...,...
11503,55536_186,-2.349694,55536,*CH3,C1
11504,84011_298,-2.974431,84011,*CHCO,C2
11505,64542_152,0.239314,64542,*CH,C1
11506,71806_221,-1.262554,71806,*CHOH,C1


In [10]:
sorted_df = df_data.sort_values(by='sid')
sorted_df

,id,target,sid,ads
7213,0_100,0.172517,0,*C*C
8628,10005_122,-0.995634,10005,*CHOH
1816,10009_220,-3.775969,10009,*NH2N(CH3)2
3377,10014_267,-2.868026,10014,*COCH2OH
10534,10017_-1,-3.603502,10017,*CH2*O
...,...,...,...,...
2100,996_174,-4.008671,996,*NH2N(CH3)2
7281,9972_90,0.521928,9972,*CH
858,9986_-1,-1.160724,9986,*OCH3
10837,9987_-1,-0.469078,9987,*COCH2OH


In [12]:
df_data['sid'].duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
11503    False
11504    False
11505    False
11506    False
11507    False
Name: sid, Length: 11508, dtype: bool

In [11]:
duplicate_sids_count = df_data['sid'].duplicated().sum()
duplicate_sids_count

0

# Catalyst

In [51]:
def determine_catalyst(catalyst, target_elements=['Zr', 'Ni', 'Al']):
    """
    Determine whether "Zr", "Ni", "Al" are included in the catalyst.

    Parameters:
    catalyst (str): The catalyst string to be evaluated.

    Returns:
    str: "Zr", "Ni", "Al" if the respective element is in the catalyst,
         otherwise return None.
    """
    
    for element in target_elements:
        if element in catalyst:
            return element
    return "Others"

In [52]:
model = 'gnoc'
data_path = f'/home/jovyan/shared-scratch/jhoon/ocp2023/clip_data/oc20dense_eval_{model}_relaxed.pkl'
df_data = pd.read_pickle(data_path)

In [53]:
df_data['sid'] = df_data['id'].apply(lambda x: x.split('_')[0])
df_data['frame'] = df_data['id'].apply(lambda x: x.split('_')[1])

# check whether there are duplicated sids
duplicate_sids_count = df_data['sid'].duplicated().sum()
print('duplicate: ', duplicate_sids_count)

# Extracting 'catalyst' from 'text' using apply()
df_data['catalyst'] = df_data['text'].apply(lambda x: x.split('</s>')[1].split(' (')[0])
df_data['category'] = df_data['catalyst'].apply(determine_catalyst)


df_data = df_data.drop(columns=['text', 'chg_emb', 'sid', 'frame', 'catalyst'])
df_data

duplicate:  0


,id,target,category
0,85487_-1,-0.559899,Others
1,39118_110,-1.130786,Zr
2,2797_-1,-1.337591,Others
3,64221_170,0.885938,Others
4,44017_160,-2.090978,Ni
...,...,...,...
11503,55536_186,-2.349694,Ni
11504,84011_298,-2.974431,Zr
11505,64542_152,0.239314,Others
11506,71806_221,-1.262554,Ni


In [54]:
category_counts = df_data['category'].value_counts()

category_counts

Others    6883
Zr        2358
Ni        2026
Al         241
Name: category, dtype: int64

In [28]:
# check whether there are duplicated sids
duplicate_sids_count = df_data['sid'].duplicated().sum()
duplicate_sids_count

0

In [ ]:
sid_list, ads_list = [], []
for i in range(len(df_data)):
    sid = df_data['id'].iloc[i].split('_')[0]
    ads = mapping[int(sid)]['adsorbate']
    sid_list.append(sid)
    ads_list.append(ads)
df_data['sid'] = sid_list
df_data['ads'] = ads_list
df_data = df_data.drop(columns=['text', 'chg_emb'])